# 한신대학교 e-비즈니스학과 201646006 이성범 PD학기제

# 앙상블 이란?

앙상블이란 간단하게 설명하자면 수천명의 사람에게 질문을 하고 얻은 대답을 통해서 결론을 내리는 방식이다. 위키백과의 집단지성과 같은 느낌이라고 할 수 있다. 위키백과가 집단지성의 힘을 통해서 전문가의 답변을 뛰어넘는 것 처럼 앙상블도 여러 종류의 모델의 결과를 모아서 하나의 전문가 모델의 성능을 뛰어넘는다.

현재 캐글 등 데이터 분석 커뮤니티에서 많이 횔용되고 있는 알고리즘들 또한 앙상블 알고리즘을 통해서 만들어진 모델들이다. RandomForest, XGBoost, LightGBM, CatBoost 등 약분류기를 여러개 모아서 하나의 강한 분류기로 만드는 앙상블 알고리즘은 정형 데이터에 대해서는 딥러닝보다 더 높은 성능을 보여주며 많이 활용되고 있다. 뿐만아니라 앙상블 기법을 통해서 만들어진 알고리즘을 다시 앙상블의 하나의 방법인 Stacking을 통해서 결과를 취합하여 더 높은 성과를 내고 있다.

하지만 집단지성이 무조건 한명의 완벽한 전문가를 이긴다는 보장은 없기 때문에 앙상블 기법을 통해서 만들어진 알고리즘이 무조건 다른 모델들 보다 성능이 높다고는 보장할 수 없다. NFL이론 처럼 다양한 모델을 활용해보고 그 중에 최고의 성능을 보이는 모델을 선택하는 것이 최선의 방법일 것이다. 또한 현업에서는 앙상블 기법을 활용하기에는 시간과 비용이 상대적으로 많이 들기 때문에 많이 활용되지는 않는다. 왜냐하면 최적의 앙상블 기법 활용 방법은 비슷한 성능을 보이는 모델들을 취합하는 것이 좋은데 비슷한 성능을 보이는 모델들을 구축하는데에는 많은 시간과 노력이 들고 들인 시간과 노력 만큼 앙상블 기법이 극적인 성능향상을 보이는 것은 아니기 때문에 효율성과 효과성을 중요시하는 기업에게는 그렇게 좋은 선택지가 되지는 않는다, 차라리 빠르게 모델을 만들고 출시를 한 후 피드백을 통해서 모델을 개선시키는 것이 기업의 입장에서는 더 나은 방식이 될 것이다.

### 앙상블의 종류
- Voting
- Bagging
- Boosting
- Stacking

# Voting

보팅은 말 그대로 투표를 통해서 결과를 취합하는 방식이다. 여러개의 서로다른 모델이 투표 또는 확률 값의 평균을 통해서 더 많은 득표를 받은, 더 높은 확률을 가지는 값을 예측하는 방식으로 크게 Hard Voting과 Soft Voting으로 나뉘어 진다.

- Hard Voting은 다수결의 원칙과 비슷한 방식으로 다수의 분류기가 예측을 한 값을 최종 예측 값으로 선택하는 방식이다.

- Soft Voting은 클래스의 결정 확률, 회귀의 예측 값등을 평균하여 최종 예측을 하는 방식이다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

iris_data = load_iris()

X = iris_data.data # 학습할 데이터
y = iris_data.target # 타겟

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22, stratify = y)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

lr_clf = LogisticRegression()
rf_cif = RandomForestClassifier(random_state = 22)
svm_clf = SVC()

voting_clf = VotingClassifier(estimators = [('lr', lr_clf), ('rf', rf_cif), ('svm', svm_clf)],
                              voting = 'hard')

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (lr_clf, rf_cif, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9666666666666667
RandomForestClassifier 0.9333333333333333
SVC 1.0
VotingClassifier 0.9666666666666667


위 처럼 무조건 앙상블을 통한 예측이 성능이 높다는 것을 보장할 수는 없지만 앙상블 방식 자체가 다수의 지식을 모으는 방식이기 때문에 과적합을 방지해주며 분산이나 편향을 줄여줄 수 있다는 장점을 가진다.

# Bagging

Bagging은 Bootstrap Aggregating의 줄임말로 훈련 세트에서 중복을 허용하여 샘플링하여 모델을 학습시키는 방식이다.

부트스트래핑은 전체에 훈련 세트에서 여러 번 복원 추출하는 샘플 카피 기법을 의미하며 카피된 샘플들을 부트스트랩 샘플이라고 한다. 이 부트스트랩 샘플을 이용해서 같은 모델을 여러개 학습시켜서 Voting 방식을 통해서 최종 예측 값을 얻는다. 범주형 변수의 경우에는 다수결로, 연속형 변수의 경우에는 평균으로 집계한다.

간단히 요약하자면 배깅은 서로 다른 훈련 샘플로 학습된 같은 모델을 Voting 방식을 통해서 결과를 취합하는 앙상블 방식이라고 할 수 있으며 배깅의 대표적인 알고리즘이 바로 RandomForest 이다.

RandomForest는 부트스트랩 샘플로 병렬 학습을 하여 최종 예측 값을 얻는 모델로 하나의 결정트리가 가지는 가장 안좋은 단점인 과적합을 여러개의 결정트리가 모여서 개선하여 대부분의 데이터에 높은 성능을 보이는 모델이다. 현재 RandomForest는 캐글, 현업 등 다양한 곳에서 많이 활용되는 알고리즘이다.

# Boosting

부스팅 방식은 배깅의 병렬 학습과 달리 순차 학습을 통해 더 나은 예측을 하는 앙상블 방법이다. 

약학습기를 순차적으로 학습하여 예측을 반복하면서 잘못 예측한 데이터에 가중치를 부여하여 오류를 개선하는 것이 부스팅 방법의 아이디어 이다.

배깅처럼 약분류기를 여러개 모아서 강분류기로 만든다는 아이디어는 동일하지만 가중치를 부여하여 학습을 한다는 점에서 배깅과의 차이점을 보이고 배깅이 부트스트랩 샘플 데이터만을 학습에 사용하는 것과 달리 부스팅은 모든 학습 데이터를 사용한다는 점에서도 배깅과 다르다.

부스팅 방식의 대표적인 알고리즘은 크게 AdaBoost와 Gradient Boost로 나뉘어진다. 에이다 부스트 같은 경우에는 잘 못 분류한 값에 가중치를 부여하면서 순차적으로 학습하여 최적의 모델을 찾는 방식이고 그래디언드 부스트 같은 경우에는 에이다 부스트와 유사하지만 가중치 업데이트로 경사하강법을 이용하는 것이 큰 차이 이다. 예측 값과 실제 값의 차이를 기반으로 가중치를 업데이트 해가면서 이전 모델에서의 오류를 다음 모델에서 줄일 수 있도록 학습하는 방식이다.

AdaBoost와 Gradient Boost 같은 경우에는 성능은 좋지만 순차 학습에 의하여 속도가 매우 느리다는 단점을 가진다. 따라서 모델의 속도를 개선시키기 위해 GPU를 활용할 수 있는 GBM의 개선된 형태인 XGBoost, LightGBM, CatBoost 등이 많이 활용되며 그 성능 또한 매우 좋다. 캐글 등 데이터 경연대회의 우승 코드를 보면 대부분 XGBoost, LightGBM, CatBoost 등이 활용되었다는 것을 쉽게 찾아볼 수 있다.

# Stacking

스택킹은 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 모든 예측기의 결과를 취합하는 모델을 만들 수는 없을까라는 의문에서 시작한 알고리즘이다. 두 개 이상의 다른 모델들을 조합해서 최고의 성능을 내는 모델을 만드는 앙상블 방법이다. 각 모델의 장점은 높이고 약점은 서로 보완해나가는 방식이다.

서로 동일한 데이터로 학습된 여러 개의 개별 모델의 예측 결과를 메타모델(예측 결과를 취합하는 모델)의 학습 데이터로 활용하여 최종 예측 결과를 내는 방식이다.

실제로 캐글 등의 데이터 경연대회를 보면 서로 다른 모델을 스택킹하여 우승한 코드들을 많이 찾아볼 수 있다.

# 앙상블 실습

앙상블 실습은 Bagging, Boosting, Stacking을 가지고 진행될 예정이며 Google Play Store의 평점을 예측하는 실습을 진행할 예정이다.

# 참고자료

- 파이썬 머신러닝 완벽 가이드, 권철민
- 파이썬 딥러닝 파이토치, 이경택
- 핸즈온 머신러닝, 박해선
- 데이터 사이언스 스쿨, https://datascienceschool.net/
- 데이터 분석 대표 대학생 연합 동아리 Tobigs, http://www.datamarket.kr/
- DeepLearnig.AI Course 1 ~ 5, Andrew Ng, https://www.youtube.com/c/Deeplearningai/playlists
- 모두를 위한 딥러닝 시즌 2, https://deeplearningzerotoall.github.io/season2/